In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
import matplotlib.cm as cm
import cPickle as pickle
import helpers
import plots
import pandas as pd
import glob
reload(plots)
reload(helpers)

<module 'helpers' from 'helpers.pyc'>

# Figure 1 notebook

## Figure 1B: Stomach fullness is highly indicative of feeding behaviour and can be used to predict the intermeal interval

## Figure 1C: Model fit to microscopic feeding behaviour yields long-term calorie intakes that match those observed across a range of nutritional states, anorectic drugs and photoperiods